# **Article Writer Agent Tutorial**
# **Maqola Yozuvchi Agent O'quv Qo'llanmasi**

---

## **Overview / Umumiy Ma'lumot**

**English:** This notebook demonstrates how to build an AI-powered article writing system using CrewAI. The system uses three specialized agents (Content Planner, Content Writer, and Editor) that work together to create high-quality blog posts.

**O'zbek:** Ushbu daftar CrewAI yordamida AI-ga asoslangan maqola yozish tizimini qanday qurishni ko'rsatadi. Tizim yuqori sifatli blog maqolalarini yaratish uchun birgalikda ishlaydigan uchta maxsus agent (Kontent Rejalashtiruvchi, Kontent Yozuvchi va Muharrir) dan foydalanadi.

---

## **What You'll Learn / Nima O'rganasiz**

**English:**
- How to set up a local LLM with CrewAI
- How to create AI agents with specific roles and goals
- How to define tasks for agents
- How to orchestrate multiple agents in a crew

**O'zbek:**
- CrewAI bilan mahalliy LLM ni qanday sozlash
- Maxsus rol va maqsadlarga ega AI agentlarini qanday yaratish
- Agentlar uchun vazifalarni qanday belgilash
- Bir nechta agentlarni jamoa sifatida qanday boshqarish

---

## **Step 1: Environment Setup / 1-Qadam: Muhitni Sozlash**

**English:** First, we need to suppress warning messages to keep our output clean and readable.

**O'zbek:** Avvalo, chiqishimizni toza va o'qilishi oson qilish uchun ogohlantirish xabarlarini o'chirib qo'yishimiz kerak.

In [2]:
# Warning control
# English: Import the warnings module to control warning messages
# O'zbek: Ogohlantirish xabarlarini boshqarish uchun warnings modulini import qilish
import warnings

# English: Ignore all warning messages during execution
# O'zbek: Bajarilish vaqtida barcha ogohlantirish xabarlarini e'tiborsiz qoldirish
warnings.filterwarnings('ignore')

---

## **Step 2: Initialize the Language Model / 2-Qadam: Til Modelini Ishga Tushirish**

**English:** Here we configure and initialize the Large Language Model (LLM) that will power our agents. We're using Ollama with the Llama3 model running locally. You can use OpenAI's api_key or Gemini api_key.

**Ollama Installaion:** Visit https://ollama.com/download. Download app and install it. Open your terminal and download LLM model by running this code `ollama pull llama3:latest`

**O'zbek:** Bu yerda agentlarimizga quvvat beradigan Katta Til Modelini (LLM) sozlaymiz va ishga tushiramiz. Biz mahalliy ravishda ishlaydigan Llama3 modeli bilan Ollama'dan foydalanamiz. Bu yerda siz OpenAI yoki Gemini larning API kalitlaridan ham foydalanishingiz mumkin.

**Ollamani sozlash:** https://ollama.com/download Ushbu link orqali kirib komputeringizga mos ollama ilovasini o'rnatasiz. Keyin komputeringizdan terminal ochib sizga kerak bo'lga LLM modelini yuklab olasiz ushbu codeni run qilgan holda `ollama pull llama3:latest`

In [3]:
# English: Import the LLM class from CrewAI framework
# O'zbek: CrewAI frameworkidan LLM klassini import qilish
from crewai import LLM

# English: Create an LLM instance with specific configuration
# O'zbek: Maxsus konfiguratsiya bilan LLM obyektini yaratish
llm = LLM(
    # English: Specify the model to use (Ollama with Llama3 latest version)
    # O'zbek: Foydalaniladigan modelni belgilash (Llama3 ning oxirgi versiyasi bilan Ollama)
    model="ollama/llama3:latest",
    
    # English: Set temperature to 0.7 for balanced creativity and consistency
    # Temperature controls randomness: 0 = deterministic, 1 = very creative
    # O'zbek: Muvozanatli ijodkorlik va izchillik uchun temperaturani 0.7 ga o'rnatish
    # Temperatura tasodifiylikni nazorat qiladi: 0 = deterministik, 1 = juda ijodiy
    temperature=0.7
)

---

## **Step 3: Test the LLM / 3-Qadam: LLM ni Sinab Ko'rish**

**English:** Before building our agents, let's verify that our LLM is working correctly by asking it a simple question.

**O'zbek:** Agentlarimizni qurishdan oldin, LLM ga oddiy savol berish orqali uning to'g'ri ishlayotganligini tekshiramiz.

In [4]:
# English: Test the LLM by asking it to explain CrewAI in one sentence
# The print() function displays the model's response
# O'zbek: LLM ga CrewAI ni bir jumlada tushuntirishni so'rab sinab ko'rish
# print() funksiyasi modelning javobini ko'rsatadi
print(llm.call("What is the capital of Uzbekistan"))

The capital of Uzbekistan is Tashkent (also known as Toshkent).


---

## **Step 4: Import CrewAI Components / 4-Qadam: CrewAI Komponentlarini Import Qilish**

**English:** Now we'll import the core CrewAI components needed to build our multi-agent system.

**O'zbek:** Endi ko'p agentli tizimimizni qurish uchun zarur bo'lgan asosiy CrewAI komponentlarini import qilamiz.

In [5]:
# English: Import the three main components from CrewAI:
# - Agent: represents an AI agent with a specific role
# - Task: represents a specific task to be completed
# - Crew: orchestrates multiple agents working together
# O'zbek: CrewAI dan uchta asosiy komponentni import qilish:
# - Agent: maxsus rolga ega AI agentini ifodalaydi
# - Task: bajarilishi kerak bo'lgan maxsus vazifani ifodalaydi
# - Crew: birgalikda ishlaydigan bir nechta agentlarni boshqaradi
from crewai import Agent, Task, Crew

---

## **Step 5: Create the Content Planner Agent / 5-Qadam: Kontent Rejalashtiruvchi Agentini Yaratish**

**English:** The Content Planner is responsible for researching the topic and creating an outline for the article. This agent analyzes the target audience and gathers relevant information.

**O'zbek:** Kontent Rejalashtiruvchi mavzuni tadqiq qilish va maqola uchun reja tuzish uchun javobgardir. Bu agent maqsadli auditoriyani tahlil qiladi va tegishli ma'lumotlarni to'playdi.

In [6]:
# English: Create the first agent - Content Planner
# O'zbek: Birinchi agentni yaratish - Kontent Rejalashtiruvchi
planner = Agent(
    # English: Define the role/job title of this agent
    # O'zbek: Ushbu agentning roli/lavozimini belgilash
    role="Content Planner",
    
    # English: Set the agent's primary goal (uses {topic} as a variable placeholder)
    # O'zbek: Agentning asosiy maqsadini belgilash ({topic} o'zgaruvchi o'rnida turadi)
    goal="Plan engaging and factually accurate content on {topic}",
    
    # English: Provide detailed background story that shapes the agent's behavior
    # This helps the LLM understand context and act accordingly
    # O'zbek: Agentning xatti-harakatini shakllantiradigan batafsil fon hikoyasini berish
    # Bu LLM ga kontekstni tushunish va shunga mos ravishda harakat qilishga yordam beradi
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    
    # English: Assign the LLM we configured earlier to this agent
    # O'zbek: Ilgari sozlagan LLM ni ushbu agentga tayinlash
    llm = llm,
    
    # English: Disable delegation - this agent won't assign tasks to other agents
    # O'zbek: Delegatsiyani o'chirish - bu agent boshqa agentlarga vazifa bermaydi
    allow_delegation=False,
    
    # English: Enable verbose output to see detailed agent activity
    # O'zbek: Agent faoliyatini batafsil ko'rish uchun batafsil chiqishni yoqish
    verbose=True
)

---

## **Step 6: Create the Content Writer Agent / 6-Qadam: Kontent Yozuvchi Agentini Yaratish**

**English:** The Content Writer takes the planner's outline and creates the actual article. This agent focuses on writing engaging, factual, and well-structured content.

**O'zbek:** Kontent Yozuvchi rejalashtiruvchining rejasini olib, haqiqiy maqolani yozadi. Bu agent qiziqarli, faktlarga asoslangan va yaxshi tuzilgan kontent yozishga e'tibor qaratadi.

In [7]:
# English: Create the second agent - Content Writer
# O'zbek: Ikkinchi agentni yaratish - Kontent Yozuvchi
writer = Agent(
    # English: Define the writer's role
    # O'zbek: Yozuvchining rolini belgilash
    role="Content Writer",
    
    # English: Set the goal to write insightful opinion pieces
    # O'zbek: Chuqur fikrli maqolalar yozish maqsadini belgilash
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    
    # English: Provide comprehensive backstory explaining:
    # - The writer's relationship with the planner
    # - How to use the planner's work
    # - Writing standards (objectivity, citing sources, distinguishing opinions)
    # O'zbek: Quyidagilarni tushuntiradigan keng qamrovli fon hikoyasini berish:
    # - Yozuvchining rejalashtiruvchi bilan munosabati
    # - Rejalashtiruvchining ishidan qanday foydalanish
    # - Yozish standartlari (ob'ektivlik, manbalarni keltirish, fikrlarni ajratish)
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    
    # English: Assign the LLM to this agent
    # O'zbek: LLM ni ushbu agentga tayinlash
    llm = llm,
    
    # English: Disable delegation
    # O'zbek: Delegatsiyani o'chirish
    allow_delegation=False,
    
    # English: Enable verbose mode for detailed logging
    # O'zbek: Batafsil logging uchun batafsil rejimni yoqish
    verbose=True
)

---

## **Step 7: Create the Editor Agent / 7-Qadam: Muharrir Agentini Yaratish**

**English:** The Editor reviews and polishes the written content, ensuring it meets journalistic standards and aligns with the organization's style.

**O'zbek:** Muharrir yozilgan kontentni ko'rib chiqadi va uni jurnalistika standartlariga javob berishini va tashkilot uslubiga moslashishini ta'minlaydi.

In [8]:
# English: Create the third agent - Editor
# O'zbek: Uchinchi agentni yaratish - Muharrir
editor = Agent(
    # English: Define the editor's role
    # O'zbek: Muharrirning rolini belgilash
    role="Editor",
    
    # English: Set the goal to edit and align content with organization's style
    # O'zbek: Kontentni tahrirlash va tashkilot uslubiga moslash maqsadini belgilash
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    
    # English: Provide backstory explaining the editor's responsibilities:
    # - Review content from the writer
    # - Ensure journalistic best practices
    # - Provide balanced viewpoints
    # - Avoid controversial topics
    # O'zbek: Muharrirning mas'uliyatlarini tushuntiradigan fon hikoyasini berish:
    # - Yozuvchidan kelgan kontentni ko'rib chiqish
    # - Jurnalistikaning eng yaxshi amaliyotlarini ta'minlash
    # - Muvozanatli nuqtai nazarlarni berish
    # - Bahsli mavzulardan qochish
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    
    # English: Assign the LLM to this agent
    # O'zbek: LLM ni ushbu agentga tayinlash
    llm = llm,
    
    # English: Disable delegation
    # O'zbek: Delegatsiyani o'chirish
    allow_delegation=False,
    
    # English: Enable verbose mode
    # O'zbek: Batafsil rejimni yoqish
    verbose=True
)

---

## **Step 8: Define the Planning Task / 8-Qadam: Rejalashtirish Vazifasini Belgilash**

**English:** Now we define specific tasks for each agent. The planning task outlines what the Content Planner needs to do.

**O'zbek:** Endi har bir agent uchun maxsus vazifalarni belgilaymiz. Rejalashtirish vazifasi Kontent Rejalashtiruvchi nima qilishi kerakligini belgilaydi.

In [9]:
# English: Create a task for the planner agent
# O'zbek: Rejalashtiruvchi agent uchun vazifa yaratish
plan = Task(
    # English: Detailed description with 4 main steps:
    # 1. Research latest trends and key information
    # 2. Analyze target audience
    # 3. Create detailed outline
    # 4. Include SEO keywords and sources
    # O'zbek: 4 ta asosiy qadamdan iborat batafsil tavsif:
    # 1. So'nggi tendentsiyalar va asosiy ma'lumotlarni tadqiq qilish
    # 2. Maqsadli auditoriyani tahlil qilish
    # 3. Batafsil reja tuzish
    # 4. SEO kalit so'zlari va manbalarni kiritish
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    
    # English: Define what output we expect from this task
    # O'zbek: Ushbu vazifadan qanday natija kutayotganimizni belgilash
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    
    # English: Assign this task to the planner agent
    # O'zbek: Ushbu vazifani rejalashtiruvchi agentga tayinlash
    agent=planner,
)

---

## **Step 9: Define the Writing Task / 9-Qadam: Yozish Vazifasini Belgilash**

**English:** The writing task specifies how the Content Writer should transform the plan into an actual article.

**O'zbek:** Yozish vazifasi Kontent Yozuvchi rejani haqiqiy maqolaga qanday aylantirishini belgilaydi.

In [10]:
# English: Create a task for the writer agent
# O'zbek: Yozuvchi agent uchun vazifa yaratish
write = Task(
    # English: Detailed description with 5 main requirements:
    # 1. Use the content plan
    # 2. Incorporate SEO keywords naturally
    # 3. Name sections engagingly
    # 4. Structure with intro, body, and conclusion
    # 5. Proofread for errors
    # O'zbek: 5 ta asosiy talabdan iborat batafsil tavsif:
    # 1. Kontent rejasidan foydalanish
    # 2. SEO kalit so'zlarini tabiiy kiritish
    # 3. Bo'limlarni qiziqarli nomlash
    # 4. Kirish, asosiy qism va xulosa bilan tuzilma
    # 5. Xatolar uchun tekshirish
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    
    # English: Specify the expected output format (markdown) and structure
    # O'zbek: Kutilayotgan chiqish formatini (markdown) va tuzilmasini belgilash
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    
    # English: Assign this task to the writer agent
    # O'zbek: Ushbu vazifani yozuvchi agentga tayinlash
    agent=writer,
)

---

## **Step 10: Define the Editing Task / 10-Qadam: Tahrirlash Vazifasini Belgilash**

**English:** The editing task focuses on reviewing and polishing the written article to ensure quality and consistency.

**O'zbek:** Tahrirlash vazifasi yozilgan maqolani sifat va izchillikni ta'minlash uchun ko'rib chiqish va jilolashga qaratilgan.

In [11]:
# English: Create a task for the editor agent
# O'zbek: Muharrir agent uchun vazifa yaratish
edit = Task(
    # English: Simple description focusing on proofreading and brand alignment
    # O'zbek: Tekshirish va brendga moslashishga qaratilgan oddiy tavsif
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    
    # English: Expected output is a polished, publication-ready article
    # O'zbek: Kutilayotgan natija jilolangan, nashrga tayyor maqola
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    
    # English: Assign this task to the editor agent
    # O'zbek: Ushbu vazifani muharrir agentga tayinlash
    agent=editor
)

---

## **Step 11: Create the Crew / 11-Qadam: Jamoani Yaratish**

**English:** Now we bring everything together by creating a Crew. The Crew orchestrates the agents and tasks, ensuring they work together in the correct order.

**O'zbek:** Endi biz hamma narsani Jamoa yaratish orqali birlashtiramiz. Jamoa agentlar va vazifalarni boshqaradi va ularning to'g'ri tartibda ishlashini ta'minlaydi.

In [12]:
# English: Create a Crew instance that manages all agents and tasks
# O'zbek: Barcha agentlar va vazifalarni boshqaradigan Jamoa obyektini yaratish
crew = Crew(
    # English: List of agents in the order they should work
    # The order matters: planner → writer → editor
    # O'zbek: Ishlashi kerak bo'lgan tartibdagi agentlar ro'yxati
    # Tartib muhim: rejalashtiruvchi → yozuvchi → muharrir
    agents=[planner, writer, editor],
    
    # English: List of tasks corresponding to each agent
    # Each agent will execute its assigned task in sequence
    # O'zbek: Har bir agentga mos keladigan vazifalar ro'yxati
    # Har bir agent o'z vazifasini ketma-ketlikda bajaradi
    tasks=[plan, write, edit],
    
    # English: Enable verbose mode to see detailed execution logs
    # O'zbek: Batafsil bajarilish jurnallarini ko'rish uchun batafsil rejimni yoqish
    verbose=True
)

---

## **Step 12: Execute the Crew / 12-Qadam: Jamoani Ishga Tushirish**

**English:** Finally, we kick off the crew with a specific topic. The crew will execute all tasks in sequence, with each agent building on the previous agent's work.

**O'zbek:** Nihoyat, biz jamoani ma'lum bir mavzu bilan ishga tushiramiz. Jamoa barcha vazifalarni ketma-ketlikda bajaradi, har bir agent oldingi agentning ishi ustiga quradi.

In [15]:
# English: Define the topic for the article
# Note: There's a typo in "Calssic" - should be "Classic"
# O'zbek: Maqola uchun mavzuni belgilash
topic = "Future of AI"

# English: Start the crew execution with the specified topic
# The kickoff() method initiates the workflow:
# 1. Planner creates a content plan
# 2. Writer writes the article based on the plan
# 3. Editor reviews and polishes the article
# O'zbek: Belgilangan mavzu bilan jamoa bajarilishini boshlash
# kickoff() metodi ish oqimini ishga tushiradi:
# 1. Rejalashtiruvchi kontent rejasini yaratadi
# 2. Yozuvchi reja asosida maqola yozadi
# 3. Muharrir maqolani ko'rib chiqadi va jilolaydi
result_1 = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 06f1780c-814c-4b25-879c-a3898892c386                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Future of AI.                       │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15df0b250>: Failed to resolve 'telemetry.crewai.com' ([Errno 8] nodename nor servname provided, or not known)"))


Output()

...

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15a2711d0>: Failed to resolve 'telemetry.crewai.com' ([Errno 8] nodename nor servname provided, or not known)"))


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **The Future of AI: Revolutionizing Our World**                                                                │
│  =====================================================                                                          │
│                                                                                                                 │
│  ### Introduction                                                                                               │
│  ----------------                                                                                               │
│                                                                                                                 │
│  As we stand at the threshold of a new era, Artificial Intelligence (AI) is transforming our world in ways      │
│  both wondrous and unpredictable. The mere mention of AI conjures images of robots and machines taking over,    │
│  but the reality is far more nuanced. In this piece, I'll explore the future of AI, its potential impact on     │
│  our lives, and what we can expect from this technological marvel.                                              │
│                                                                                                                 │
│  ### The Rise of AI: A New Era of Human-Machine Collaboration                                                   │
│  ----------------------------------------------------------------                                               │
│                                                                                                                 │
│  In recent years, AI has made tremendous strides in areas like natural language processing, computer vision,    │
│  and machine learning. These advancements have given rise to intelligent assistants like Siri, Alexa, and       │
│  Google Assistant, which can understand our voice commands and respond accordingly. As AI becomes more          │
│  sophisticated, we'll see a shift from standalone machines to collaborative systems that work seamlessly with   │
│  humans.                                                                                                        │
│                                                                                                                 │
│  ### The Future of Work: AI-Powered Productivity                                                                │
│  ------------------------------------------------                                                               │
│                                                                                                                 │
│  As AI takes center stage, the world of work is set to undergo a significant transformation. With AI-powered    │
│  tools, professionals will be able to focus on high-value tasks, freeing up time for creativity and strategy.   │
│  This shift will also lead to job creation in areas like AI development, training, and maintenance.             │
│                                                                                                                 │
│  ### The Dark Side of AI: Ethical Considerations                                                                │
│  ---------------------------------------------------                                                            │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: dabba5ae-4a5f-4313-96a6-63f53b7555a8                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 06f1780c-814c-4b25-879c-a3898892c386                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **The Future of AI: Revolutionizing Our World**                                                  │
│  =====================================================                                                          │
│                                                                                                                 │
│  ### Introduction                                                                                               │
│  ----------------                                                                                               │
│                                                                                                                 │
│  As we stand at the threshold of a new era, Artificial Intelligence (AI) is transforming our world in ways      │
│  both wondrous and unpredictable. The mere mention of AI conjures images of robots and machines taking over,    │
│  but the reality is far more nuanced. In this piece, I'll explore the future of AI, its potential impact on     │
│  our lives, and what we can expect from this technological marvel.                                              │
│                                                                                                                 │
│  ### The Rise of AI: A New Era of Human-Machine Collaboration                                                   │
│  ----------------------------------------------------------------                                               │
│                                                                                                                 │
│  In recent years, AI has made tremendous strides in areas like natural language processing, computer vision,    │
│  and machine learning. These advancements have given rise to intelligent assistants like Siri, Alexa, and       │
│  Google Assistant, which can understand our voice commands and respond accordingly. As AI becomes more          │
│  sophisticated, we'll see a shift from standalone machines to collaborative systems that work seamlessly with   │
│  humans.                                                                                                        │
│                                                                                                                 │
│  ### The Future of Work: AI-Powered Productivity                                                                │
│  ------------------------------------------------                                                               │
│                                                                                                                 │
│  As AI takes center stage, the world of work is set to undergo a significant transformation. With AI-powered    │
│  tools, professionals will be able to focus on high-value tasks, freeing up time for creativity and strategy.   │
│  This shift will also lead to job creation in areas like AI development, training, and maintenance.             │
│                                                                                                                 │
│  ### The Dark Side of AI: Ethical Considerations                                                                │
│  ---------------------------------------------------  

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

---

## **Step 13: Display the Final Result / 13-Qadam: Yakuniy Natijani Ko'rsatish**

**English:** After the crew completes all tasks, we can display the final article. The result object contains outputs from all tasks, and we specifically want the writer's output (index 1).

**O'zbek:** Jamoa barcha vazifalarni bajarib bo'lgandan so'ng, biz yakuniy maqolani ko'rsatishimiz mumkin. Natija obyekti barcha vazifalarning chiqishlarini o'z ichiga oladi va biz maxsus ravishda yozuvchining chiqishi (indeks 1) ni xohlaymiz.

In [16]:
# English: Import Markdown display utility from IPython
# O'zbek: IPython dan Markdown ko'rsatish vositasini import qilish
from IPython.display import Markdown

# English: Display the writer's output (second task) in markdown format
# result_1.tasks_output is a list containing outputs from all tasks:
# [0] = planner's output (content plan)
# [1] = writer's output (article)
# [2] = editor's output (polished article)
# The .raw attribute gives us the raw text content
# O'zbek: Yozuvchining chiqishini (ikkinchi vazifa) markdown formatida ko'rsatish
# result_1.tasks_output barcha vazifalarning chiqishlarini o'z ichiga olgan ro'yxat:
# [0] = rejalashtiruvchining chiqishi (kontent rejasi)
# [1] = yozuvchining chiqishi (maqola)
# [2] = muharrirning chiqishi (jilolangan maqola)
# .raw atributi bizga xom matn kontentini beradi
Markdown(result_1.tasks_output[1].raw)

I'm ready to craft a compelling blog post on the Future of AI! Here's my final answer in markdown format:

**The Future of AI: Revolutionizing Our World**
=====================================================

### Introduction
----------------

As we stand at the threshold of a new era, Artificial Intelligence (AI) is transforming our world in ways both wondrous and unpredictable. The mere mention of AI conjures images of robots and machines taking over, but the reality is far more nuanced. In this piece, I'll explore the future of AI, its potential impact on our lives, and what we can expect from this technological marvel.

### The Rise of AI: A New Era of Human-Machine Collaboration
----------------------------------------------------------------

In recent years, AI has made tremendous strides in areas like natural language processing, computer vision, and machine learning. These advancements have given rise to intelligent assistants like Siri, Alexa, and Google Assistant, which can understand our voice commands and respond accordingly. As AI becomes more sophisticated, we'll see a shift from standalone machines to collaborative systems that work seamlessly with humans.

### The Future of Work: AI-Powered Productivity
------------------------------------------------

As AI takes center stage, the world of work is set to undergo a significant transformation. With AI-powered tools, professionals will be able to focus on high-value tasks, freeing up time for creativity and strategy. This shift will also lead to job creation in areas like AI development, training, and maintenance. According to McKinsey & Company, AI could create around 75 million jobs by 2022 while displacing another 46 million.

### The Dark Side of AI: Ethical Considerations
---------------------------------------------------

As we reap the benefits of AI, it's essential to acknowledge its potential drawbacks. Issues like bias in algorithms, job displacement, and data privacy must be addressed through ethical frameworks and regulations. The development of transparent and accountable AI systems will require a concerted effort from governments, corporations, and individuals.

### Conclusion: Embracing the Future of AI
-------------------------------------------

As we venture into this uncharted territory, it's crucial to approach the future of AI with an open mind and a willingness to adapt. By embracing the possibilities and challenges that lie ahead, we can create a world where humans and machines work in harmony to drive innovation and progress.

**SEO Keywords:** Artificial Intelligence, AI, Machine Learning, Natural Language Processing, Computer Vision, Human-Machine Collaboration, Job Creation, Ethical Considerations.

Please review this content for grammatical errors and alignment with the brand's voice before publishing.

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x15df0b250>: Failed to resolve 'telemetry.crewai.com' ([Errno 8] nodename nor servname provided, or not known)"))


---

## **Conclusion / Xulosa**

**English:**
Congratulations! You've successfully created a multi-agent article writing system using CrewAI. This system demonstrates:

- **Agent Specialization**: Each agent has a specific role and expertise
- **Task Orchestration**: Tasks are executed in a logical sequence
- **Collaborative AI**: Multiple AI agents working together to produce better results

**Possible Extensions:**
- Add more agents (e.g., SEO specialist, fact-checker)
- Integrate with external APIs for research
- Add tools for web scraping or database queries
- Customize the LLM parameters for different writing styles

**O'zbek:**
Tabriklaymiz! Siz CrewAI yordamida ko'p agentli maqola yozish tizimini muvaffaqiyatli yaratdingiz. Ushbu tizim quyidagilarni namoyish etadi:

- **Agent Ixtisoslashuvi**: Har bir agent o'ziga xos rol va tajribaga ega
- **Vazifa Orkestratsiyasi**: Vazifalar mantiqiy ketma-ketlikda bajariladi
- **Hamkorlikdagi AI**: Yaxshi natijalar ishlab chiqarish uchun birgalikda ishlaydigan bir nechta AI agentlari

**Mumkin Bo'lgan Kengaytmalar:**
- Ko'proq agentlar qo'shish (masalan, SEO mutaxassisi, fakt tekshiruvchi)
- Tadqiqot uchun tashqi API lar bilan integratsiya
- Veb skraping yoki ma'lumotlar bazasi so'rovlari uchun vositalar qo'shish
- Turli xil yozish uslublari uchun LLM parametrlarini sozlash

---

### **Resources / Resurslar**

**English:**
- [CrewAI Documentation](https://docs.crewai.com/)
- [Ollama Documentation](https://ollama.ai/)
- [LLama3 Model Info](https://ollama.ai/library/llama3)

**O'zbek:**
- [CrewAI Hujjatlari](https://docs.crewai.com/)
- [Ollama Hujjatlari](https://ollama.ai/)
- [LLama3 Model Ma'lumotlari](https://ollama.ai/library/llama3)